In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [2]:
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

HOSPITAL_A_ANALIZAR = 112103  # Este es el Torax

# 1. Análisis de Procedimientos

En este cuadernillo se quiere responder a las siguientes preguntas:

- ¿Cuáles son las intervenciones quirúrgicas realizadas en cada uno de los egresos? Además,
  ¿cuánto corresponde cada una al total de los egresos?

- ¿Cuáles son los procedimientos realizados en cada uno de los egresos? Además, ¿Cuántos corresponde cada uno al total de egresos?

Para responder estas preguntas se utilizará la base de egresos DEIS entre 2001 a 2021.


In [9]:
df_polars = (
    pl.scan_csv("../data/processed/egresos_procesados.csv")
    .filter(pl.col("ESTABLECIMIENTO_SALUD") == HOSPITAL_A_ANALIZAR)
    .collect(streaming=True)
)

df = df_polars.to_pandas()

In [17]:
resumen_interv_q = df.groupby(["ANO_EGRESO", "DIAG1"])[
    ["CODIGO_INTERV_Q_PPAL", "GLOSA_INTERV_Q_PPAL"]
].value_counts().reset_index()

resumen_procedimientos = df.groupby(["ANO_EGRESO", "DIAG1"])[
    ["CODIGO_PROCED_PPAL", "GLOSA_PROCED_PPAL"]
].value_counts().reset_index()

In [18]:
display(resumen_interv_q)
display(resumen_procedimientos)

,ANO_EGRESO,DIAG1,CODIGO_INTERV_Q_PPAL,GLOSA_INTERV_Q_PPAL,count
0,2004,A162,1704009,"Toracotomía exploradora, c/s biopsia, c/s debr...",1
1,2004,B449,1704043,Neumonectomía c/s resección de pared costal,1
2,2004,B59X,1704035,Biopsia pulmonar por toracotomía,1
3,2004,B673,1704017,Tumores o quistes de mediastino (anterior o po...,1
4,2004,B679,1704037,Cirugía de quiste hidatídico sin resección pul...,10
...,...,...,...,...,...
6231,2020,Z450,1703148,Cambio de generador de marcapaso (incluye el v...,10
6232,2020,Z450,1703153,Implantación de marcapaso c/electrod. intraven...,1
6233,2020,Z470,1704009,"Toracotomía exploradora, c/s biopsia, c/s debr...",1
6234,2020,Z470,1704004,"Reparación pectum excavatum o carinatum, (proc...",1


,ANO_EGRESO,DIAG1,CODIGO_PROCED_PPAL,GLOSA_PROCED_PPAL,count
0,2018,A099,1701037,Puncion subclavia o yugular con colocacion de ...,1
1,2018,C33X,1707022,Larigotraqueoscopia con tubo rigido,1
2,2018,C340,1707022,Larigotraqueoscopia con tubo rigido,2
3,2018,C341,1707023,Mediastinoscopia c/s biopsia,19
4,2018,C342,1707023,Mediastinoscopia c/s biopsia,1
...,...,...,...,...,...
377,2020,Z511,2904008,Quimioterapia riesgo intermedio 2 (por ciclo),1
378,2020,Z511,2904006,Quimioterapia bajo riesgo 4 (por ciclo),1
379,2020,Z511,2904004,Quimioterapia bajo riesgo 2 (por ciclo),1
380,2020,Z514,1101045,"Polisomnografia (estudio poligrafico del sueno),",1


Por lo tanto, se muestra el desglose de intervenciones quirúrgicas y procedimientos para cada
uno de los diagnósticos por año.

In [20]:
resumen_interv_q.to_csv("../data/interim/resumen_intervenciones_quir_torax.csv")
resumen_procedimientos.to_csv("../data/interim/resumen_procedimientos_torax.csv")